In [11]:
import csv
import pandas as pd

In [22]:
df = pd.read_csv('data/testset.csv', sep=';', engine='python')

In [ ]:
df_el = df.drop(columns=['prob_REF', 'prediction', 'Unnamed: 0', 'Data valutazione pratica'])

In [ ]:
df_el

In [ ]:
len1 = len(df_el)
df_el.drop_duplicates(subset='sCodiceDomandaSiel',inplace=True)
len(df_el)

In [ ]:
df_el['cliente societ� anni'].count()/len(df_el)

In [24]:
df_el

,Rischio totale LS,Domanda di finanziamento,outstanding LS,totale finanziato gruppo,totale finanziato,rating bplg - statistico -output2,rating bnp paribasoutput2,Assileaoutput2,Canale di apportooutput2,cliente societ� anni,...,Enveloppe location immobilier,Locaz. Finanziaria Immobiliare,Locazione Finanziaria,Locazione Operativa,Package credito,Package locazione,Riacquisto di crediti,Esito valutazione della pratica da parte dell'analista (IMPORTANTE),sCodiceDomandaSiel,prob_ACC
0,101402.08,80000.00,21402.08,0.00,28000.0,6,6,0.0,NaN,0.9697802197802198,...,0,0,1,0,0,0,0,ACC,W0026134,0.989272
1,20000.00,20000.00,0.00,431900.00,0.0,6,6,0.0,6.0,NaN,...,0,0,1,0,0,0,0,ACC,Y0011780,0.781086
2,48666.19,24042.20,24623.99,140401.07,25593.4,0,6,0.0,NaN,0.14560439560439561,...,0,0,0,1,0,0,0,ACC,S0009931,0.939465
3,31000.00,31000.00,0.00,0.00,0.0,6,6,0.0,NaN,NaN,...,0,0,1,0,0,0,0,ACC,P0030493,0.660786
4,19000.00,19000.00,0.00,0.00,0.0,7,6,0.0,5.0,NaN,...,0,0,1,0,0,0,0,ACC,Y0024288,0.633539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125817,10642.07,10642.07,0.00,204500.00,0.0,7,6,0.0,1.0,NaN,...,0,0,0,1,0,0,0,REF,W0059208,0.212681
125818,3639.35,3639.35,0.00,0.00,0.0,7,6,0.0,1.0,NaN,...,0,0,0,1,0,0,0,REF,W0054307,0.257946
125819,22800.00,22800.00,0.00,0.00,0.0,9,6,0.0,6.0,NaN,...,0,0,1,0,0,0,0,REF,W0024770,0.145194
125820,1050.00,1050.00,0.00,6784.73,0.0,2,6,0.0,1.0,NaN,...,0,0,0,0,0,1,0,REF,X0015169,0.553685


In [25]:
len1 = len(df_el)
df_el.drop_duplicates(subset='sCodiceDomandaSiel',inplace=True)
len(df_el)

79624

In [28]:
df_el['cliente societ� anni'].count()/len(df_el)

0.3092409323821963